In [ ]:
#Data preparation
import os
import numpy as np
import torch
from Bio.PDB import *
from Bio.PDB.DSSP import DSSP

class ProteinDataProcessor:
    def __init__(self, pesto_file_path, pdb_directory):
        self.pesto_file_path = pesto_file_path
        self.pdb_directory = pdb_directory
        self.pesto_dict = {}
        self.secondary_dict = {}
        self.ss_mapping = {'H': 0, 'G': 0, 'I': 0, 'E': 1, 'T': 2, 'B': 2, 'S': 2, '-': 3}

    def read_pesto_file(self):
        with open(self.pesto_file_path, 'r') as file:
            for line in file:
                columns = line.strip().split('\t')
                protein_name = columns[0]
                data_values = [float(value) for value in columns[1:] if value != '0']  
                self.pesto_dict[protein_name] = data_values

    def process_secondary_structure(self):
        for pdb_filename in os.listdir(self.pdb_directory):
            pdb_filepath = os.path.join(self.pdb_directory, pdb_filename)
            parser = PDBParser(QUIET=True)
            structure = parser.get_structure("protein", pdb_filepath)
            model = structure[0]
            dssp = DSSP(model, pdb_filepath)
            protein_data = []

            for residue in dssp:
                res_id = residue[1]
                ss = residue[2]
                code = self.ss_mapping[ss]
                codes = [code]  
                #one-hot encoding
                one_hot_codes = np.eye(4)[codes]  
                protein_data.append(one_hot_codes)

            protein_tensor = torch.tensor(protein_data, dtype=torch.float32).squeeze()
            protein_name = os.path.splitext(pdb_filename)[0]
            self.secondary_dict[protein_name] = protein_tensor

    def prepare_data(self):
        self.read_pesto_file()
        self.process_secondary_structure()
        return self.pesto_dict, self.secondary_dict

file_path = "/mnt/disk1/guoxiaokun/isoform/PeSTO/PeSTo-main/geometirc.allfeature.txt"
pdb_directory = "/mnt/disk1/guoxiaokun/isoform/double.GCN/PDB"

processor = ProteinDataProcessor(file_path, pdb_directory)
pesto_dict, secondary_dict = processor.prepare_data()

In [ ]:
import os
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
from torch_geometric.data import InMemoryDataset, Data
from tqdm import tqdm
from Bio.PDB import PDBParser
import numpy as np
from multiprocessing import Pool
import scipy.spatial.distance as dist
import biotite.structure.io.pdb as pdb

def get_edge(coords, protein_name, threshold, pesto_dict):
    """Compute edge connections for a protein."""
    res_probs = pesto_dict[protein_name]
    dists = dist.cdist(coords, coords)
    edges = []
    for i in range(len(dists)):
        for j in range(len(dists)):
            if res_probs[i] >= 0.1 and res_probs[j] >= 0.1 and i != j and dists[i, j] < threshold  and abs(i - j) != 1:
                edges.append((i, j))
            if i != j and dists[i, j] < threshold  and abs(i - j) != 1:
                edges.append((i, j))
    return edges

# 生成蛋白质的图数据
def generate_edge_index(esm_file, pdb_file, protein_name, threshold, pesto_dict):
    esm_feature = torch.load(esm_file)['representations'][33]
    protein_feature_esm = torch.load(esm_file)['mean_representations'][33]
    
    with open(pdb_file, 'r') as f:
        model = pdb.PDBFile.read(f).get_structure(model=1)
    x = esm_feature
    esm2 = protein_feature_esm
    coord = [i.coord for i in model if i.atom_name == "CA"]
    edge_index = torch.tensor(get_edge(coord, protein_name, threshold, pesto_dict)).T
    return Data(x=x, edge_index=edge_index, esm2=esm2)

# Generate the graph and merge features
def generate_protein_graphs(pdb_list_file, pdb_dir, esm_dir, threshold, pesto_dict, secondary_dict):
    with open(pdb_list_file, 'r') as file:
        f_names = [protein_name.rstrip() for protein_name in file]
        pdb_files = [os.path.join(pdb_dir, protein_name + '.pdb') for protein_name in f_names]
        esm_files = [os.path.join(esm_dir, protein_name + '.pt') for protein_name in f_names]
    p = Pool(5)
    result = [[protein_name, p.apply_async(generate_edge_index, (esm_file, pdb_file, protein_name, threshold, pesto_dict))] 
              for esm_file, pdb_file, protein_name in zip(esm_files, pdb_files, f_names)]
    p.close()
    p.join()
    protein_dict = {k: v.get() for (k, v) in result}
    for protein_name, data in protein_dict.items():
        esm_fea = protein_dict[protein_name].x
        protein_dict[protein_name].x = torch.cat((secondary_dict[protein_name], esm_fea), dim=1)
    
    return protein_dict

pdb_list_file = '/mnt/disk1/guoxiaokun/isoform/double.GCN/pdb.list.txt'
pdb_dir = '/mnt/disk1/guoxiaokun/isoform/double.GCN/PDB/'
esm_dir = '/mnt/disk1/guoxiaokun/isoform/double.GCN/isoform_esm2/'
threshold = 8
protein_dict = generate_protein_graphs(pdb_list_file, pdb_dir, esm_dir, threshold, pesto_dict, secondary_dict)

# The feature of protein
for protein_name, data in protein_dict.items():
    print(f"{protein_name}: {data.x.shape}")


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn import metrics
import random
import pandas as pd
import scipy.sparse as spp

class ProteinDataLoader:
    def __init__(self, protein_dict, train_file, test_file, batch_size=3, seed=2066):
        self.device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
        self.protein_dict = protein_dict
        self.batch_size = batch_size
        self.seed = seed
        self.setup_seed(self.seed)
        
        # Load train and test data
        self.train_file = train_file
        self.test_file = test_file
        self.train_df = pd.read_csv(self.train_file, sep="\t", header=None)
        self.test_df = pd.read_csv(self.test_file, sep="\t", header=None)
        
        # Prepare datasets and dataloaders
        self.test_data = self.generate_data(self.test_df)
        self.test_dataset = ProteinDataset(self.test_data)
        self.test_loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)

        self.raw_train_data = self.generate_data(self.train_df)
        self.raw_train_dataset = ProteinDataset(self.raw_train_data)
        self.raw_train_loader = DataLoader(self.raw_train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=self.collate_fn)
    
    def setup_seed(self, seed):
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
    
    def generate_data(self, df):
        """Generate data list from dataframe and protein dictionary."""
        data_list = []
        for i in range(len(df)):
            protein1, protein2, label = df.iloc[i]
            data1 = self.protein_dict[protein1]
            data2 = self.protein_dict[protein2]
            data_list.append((data1, data2, torch.tensor([label], dtype=torch.float).to(self.device)))
        return data_list

    def collate_fn(self, batch):
        """Custom collate function for DataLoader."""
        data1 = [item[0] for item in batch]
        data2 = [item[1] for item in batch]
        label = torch.stack([item[2] for item in batch])
        return [data1, data2, label]

class ProteinDataset(Dataset):
    """Custom Dataset class for protein interaction data."""
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def weights_init(m):
     if isinstance(m, (nn.Conv1d, nn.Linear)):
       nn.init.kaiming_normal_(m.weight, mode='fan_in',
                                 nonlinearity='leaky_relu')
       
train_file = 'train.txt'
test_file = 'test.txt'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# Create an instance of ProteinDataLoader
data_loader = ProteinDataLoader(protein_dict, train_file, test_file)

test_loader = data_loader.test_loader
raw_train_loader = data_loader.raw_train_loader

In [4]:
import os
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import torch.nn as nn

# GCN模型定义
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 512).to(device)
        self.conv2 = GCNConv(512, 256).to(device)
        
        self.cnn1 = torch.nn.Conv1d(1, 8, kernel_size=2, stride=1) 
        self.normal_layer3 = torch.nn.Linear(511, 511)

        self.cnn2 = torch.nn.Conv1d(8, 16, kernel_size=2, stride=1) 
        self.normal_layer4 = torch.nn.Linear(510, 510)

        self.fc1 = torch.nn.Linear(16*510, 2560).to(device)
        self.fc2 = torch.nn.Linear(2560, 512).to(device)
        self.fc3 = torch.nn.Linear(512, 128).to(device)
        self.fc4 = torch.nn.Linear(128, 1).to(device)
        self.fc5 = torch.nn.Linear(128, 1).to(device)
        self.elu = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(0.5).to(device)  # Dropout layer 0.2

    def forward(self, data1, data2):
        x1, edge_index1 = data1.x.to(device), data1.edge_index.to(device)
        x2, edge_index2 = data2.x.to(device), data2.edge_index.to(device)

        x1 = F.leaky_relu(self.conv1(x1, edge_index1))   
        x1 = F.leaky_relu(self.conv2(x1, edge_index1))
        x1 = torch.mean(x1, 0, keepdim=True)  # Average pooling

        x2 = F.leaky_relu(self.conv1(x2, edge_index2))
        x2 = F.leaky_relu(self.conv2(x2, edge_index2))
        x2 = torch.mean(x2, 0, keepdim=True)  # Average pooling
        out = torch.cat((x1,x2),dim=1)
        
        out = F.leaky_relu(self.cnn1(out))
        out = self.normal_layer3(out)
        out = F.leaky_relu(self.cnn2(out))
        out = self.normal_layer4(out).view(-1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)

        return out


In [ ]:
import torch
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score
import time
# train and test
def train_and_test(model_class, raw_train_loader, test_loader, device, num_epochs, lr, fold_count=5):
    for kf in range(fold_count):
        print(f"kf: {kf}\n")
        model = model_class(1284, 1).to(device)
        model.apply(weights_init)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

        best_auc = 0.0  
        best_model_path = f'best_model_fold{kf}.pth'  

        for epoch in range(num_epochs): 
            start = time.time() 
            model.train()
            total_loss = 0
            y_trues, y_preds = [], []

            # train dataset
            for data1, data2, label in raw_train_loader:
                data1 = [d1.to(device) for d1 in data1]
                data2 = [d2.to(device) for d2 in data2]
                out = torch.stack([model(data1[i], data2[i]) for i in range(len(data1))])
                loss = criterion(out, label)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                y_trues.extend(label.to('cpu').tolist()) 
                y_preds.extend(out.to('cpu').detach().numpy().flatten())  
                total_loss += loss.item()

            train_loss = total_loss / len(raw_train_loader)

            model.eval()
            total_loss = 0
            y_trues, y_preds = [], []
            with torch.no_grad():
                for data1, data2, label in test_loader:
                    data1 = [d1.to(device) for d1 in data1]
                    data2 = [d2.to(device) for d2 in data2]
                    out = torch.stack([model(data1[i], data2[i]) for i in range(len(data1))])
                    loss = criterion(out, label)
                    
                    y_trues.extend(label.to('cpu').tolist())  
                    y_preds.extend(out.to('cpu').detach().numpy().flatten()) 
                    total_loss += loss.item()

            val_loss = total_loss / len(test_loader)
            auc = roc_auc_score(y_trues, y_preds)
            print(f"epoch: {epoch}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}, AUC: {auc:.4f}")

            # save best model
            if auc > best_auc:
                best_auc = auc
                torch.save(model.state_dict(), best_model_path)
                print(f"New best model found with AUC: {best_auc:.4f}. Model saved to {best_model_path}")

            scheduler.step(val_loss)

        print(f"Best AUC for fold {kf}: {best_auc:.4f}\n")


train_and_test(GCN, raw_train_loader, test_loader, device, num_epochs=50, lr=0.0001)


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn import metrics
import random
import pandas as pd
import scipy.sparse as spp
from zzd.utils.assess import multi_scores as scores
class ProteinDataLoader:
    def __init__(self, protein_dict, train_file, test_file, batch_size=3, seed=2066):
        self.device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
        self.protein_dict = protein_dict
        self.batch_size = batch_size
        self.seed = seed
        self.setup_seed(self.seed)
        
        # Load train and test data
        self.train_file = train_file
        self.test_file = test_file
        self.train_df = pd.read_csv(self.train_file, sep="\t", header=None)
        self.test_df = pd.read_csv(self.test_file, sep="\t", header=None)
        
        # Prepare datasets and dataloaders no shuffle
        self.test_data = self.generate_data(self.test_df)
        self.test_dataset = ProteinDataset(self.test_data)
        self.test_loader = DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)

        self.raw_train_data = self.generate_data(self.train_df)
        self.raw_train_dataset = ProteinDataset(self.raw_train_data)
        self.raw_train_loader = DataLoader(self.raw_train_dataset, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)
    
    def setup_seed(self, seed):
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        np.random.seed(seed)
        random.seed(seed)
        torch.backends.cudnn.deterministic = True
    
    def generate_data(self, df):
        """Generate data list from dataframe and protein dictionary."""
        data_list = []
        for i in range(len(df)):
            protein1, protein2, label = df.iloc[i]
            data1 = self.protein_dict[protein1]
            data2 = self.protein_dict[protein2]
            data_list.append((data1, data2, torch.tensor([label], dtype=torch.float).to(self.device)))
        return data_list

    def collate_fn(self, batch):
        """Custom collate function for DataLoader."""
        data1 = [item[0] for item in batch]
        data2 = [item[1] for item in batch]
        label = torch.stack([item[2] for item in batch])
        return [data1, data2, label]

class ProteinDataset(Dataset):
    """Custom Dataset class for protein interaction data."""
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
       
train_file = 'train.txt'
test_file = 'test.txt'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

data_loader = ProteinDataLoader(protein_dict, train_file, test_file)
# Access the dataloaders
test_loader = data_loader.test_loader
raw_train_loader = data_loader.raw_train_loader

In [8]:
import torch
import numpy as np

def setup_model_and_predict(model_class, model_path, device, test_loader, train_loader, test_file, train_file, output_test_file, output_train_file):
    """
    Function to load a model, perform predictions on test and train data, and save the results to files.

    Parameters:
    - model_class: The model class (e.g., GCN).
    - model_path: The path to the trained model file (.pth).
    - device: The device to run the model on (e.g., 'cuda' or 'cpu').
    - test_loader: DataLoader for test data.
    - train_loader: DataLoader for train data.
    - test_file: File containing test data information.
    - train_file: File containing train data information.
    - output_test_file: File path to save test prediction results.
    - output_train_file: File path to save train prediction results.
    """
    try:
        # Load the model
        model = model_class(1284, 1).to(device)  # 1284 is the input feature dimension, 1 is the output dimension
        model.load_state_dict(torch.load(model_path)) 
        model.eval() 
        
        def predict_and_save(loader, input_file, output_file, phase):
            y_trues, y_preds = [], []
            with torch.no_grad():
                for data1, data2, label in loader:
                    data1 = [d1.to(device) for d1 in data1]
                    data2 = [d2.to(device) for d2 in data2]
                    out = []
                    for i in range(len(data1)):
                        o = model(data1[i], data2[i])
                        out.append(o)
                    out = torch.stack(out)
                    out = torch.sigmoid(out)
                    y_preds.append(out.to('cpu').detach().tolist())
                    y_trues.append(label.to('cpu').tolist())
                
            y_preds = [item for sublist in y_preds for item in sublist]
            y_pred = np.array(y_preds)
            pred_table = np.hstack((np.genfromtxt(input_file, str), y_pred.reshape(-1, 1)))
            result_score = scores(pred_table[:, -2], pred_table[:, -1], show=True)
            print(f"Scores for {phase}: {result_score}")

            with open(output_file, "w") as f:
                for i in range(len(pred_table)):
                    f.write(f"{pred_table[i, 0]}\t{pred_table[i, 1]}\t{pred_table[i, -2]}\t{pred_table[i, -1]}\n")
        
        predict_and_save(test_loader, test_file, output_test_file, "test")
        predict_and_save(train_loader, train_file, output_train_file, "train")

    except Exception as e:
        print("Error occurred during model loading or prediction:", str(e))


setup_model_and_predict(GCN, "/mnt/disk1/guoxiaokun/isoform/double.GCN/second.result.e25.p01.filteradj.fold2_best_model.pth", device, test_loader, raw_train_loader, test_file, train_file, "GCN.ESM2.test.predict.txt", "GCN.ESM2.train.predict.txt")


TP	TN	FP	FN	precision	recall	specificity	Acc	MCC	f1	AUROC	AUPRC
118	117	34	31	0.7763	0.792	0.775	0.783	0.567	0.784	0.836	0.824
Scores for test: (118, 117, 34, 31, 0.7763, 0.7919, 0.7748, 0.78333, 0.56682, 0.78405, 0.83644, 0.82429)
TP	TN	FP	FN	precision	recall	specificity	Acc	MCC	f1	AUROC	AUPRC
564	587	12	37	0.9792	0.938	0.980	0.959	0.919	0.958	0.994	0.995
Scores for train: (564, 587, 12, 37, 0.9792, 0.9384, 0.98, 0.95917, 0.91914, 0.95837, 0.99409, 0.99458)
